
# Generalized Sequential Pattern (GSP) Mining

Apriori-based method: GSP (Generalized
Sequential Patterns: Srikant & Agrawal) 


<font color='cyan'> **Steps for Implementing GSP Mining** </font>

  Generate frequent events

 Repeat until no sequences can be generated:
> **1-** Generate candidate sequences.

> **2-** Candidate Pruning.

> **3-** Calculate support.

> **4-** Support pruning.




In [1]:
import copy
import numpy as np
import pandas as pd
from operator import neg

### Dataset

| Seq. ID | Sequence |
|:--------------:|--------------:|
| 10 | <{bd}{c}{b}> |
| 20 | <{bf}{ce}{b}>|
| 30 | <{ag}{bf}> |
| 40 | <{be}{ce}> |
| 50 | <{a}{bd}{b}{c}{b}>|


In [2]:
dataset = [   
    # sequence: list of events
    # event: [list of strings]

    [['b','d'], ['c'], ['b']],  
    [['b' , 'f'], ['c','e'], ['b']], 
    [['a', 'g'], ['b','f']], 
    [['b','e'], ['c','e']],
    [['a'], ['b','d'],['b'],['c'],['b']]
]

### Definition of a Sequence

A sequence is an ordered list of elements
> s = < e1 e2 e3 … >

• Each element contains a collection of events
(items)
> ei = {i1, i2, …, ik}

• Length of a sequence, |s|, is given by the
number of elements in the sequence

• A k-sequence is a sequence that contains k
events (items)
5

***

### Subsequences
•Sequence t is a subsequence of s if each
ordered element in t is a subset of an
ordered element in s.

• Examples:

> <{1}{2}> is a subsequence of <{1}{2,3}{4}>

> <{1}{4}> is a subsequence of <{1}{2,3}{4}>

> But <{1}{2}{3}> is not a subsequence of <{1}{2,3}{4}>


Sequence A |Sequence B |Is B subsequence of A?
-----|-----|----- 
<{2,4} {3,5,6} {8}>|<{2} {3,6} {8}>|<font color='green'> **Yes** </font>
<{2,4} {3,5,6} {8}>|<{2} {8}>|<font color='green'> **Yes**  </font>
<{1,2} {3,4}>|<{1} {2}>|<font color='red'> **No**  </font>
<{2,4} {2,4} {2,5}>|<{2} {4}>|<font color='green'> **Yes**  </font>

In [3]:
def is_subsequence(main_sequence, subsequence):
  
    def is_subsequence_recursive(subsequence_clone, start=0):
        
        # check if empty: end of recursion, all itemsets have been found
        if not subsequence_clone:
            return True
        # retrieves element of the subsequence and removes it from subsequence 
        first_elem = set(subsequence_clone.pop(0))
        # search for the first itemset...
        for i in range(start, len(main_sequence)):
            if set(main_sequence[i]).issuperset(first_elem):
                # and recurse
                return is_subsequence_recursive(subsequence_clone, i + 1)
        return False

    return is_subsequence_recursive(subsequence.copy()) 

In [4]:
sequence = [['a'], ['b', 'c'], ['d'], ['a', 'e']]

In [5]:
subsequence = [['a'], ['b', 'c'], ['e']]
is_subsequence(sequence,subsequence)

True

In [6]:
subsequence = [['a'], ['b', 'd']]
is_subsequence(sequence,subsequence)

False

## Sequence Length

In [7]:
def sequence_length(sequence):
 
    return sum(len(i) for i in sequence)

In [8]:
sequence_length([['a'], ['b', 'c'], ['a'], ['b', 'c', 'd']])

7

## Timing Constraints

### Maximum Span

Maximum time between latest and earliest
events in a sequence.
> It affects the support count step.

> Example: if maxspan is 3

Sequence A |Sequence B |Is B subsequence of A?
-----|-----|----- 
<{1,3}{3,4}{4}{5}{6,7}{8}>|<{3} {4}>|<font color='green'> **Yes** </font>
<{1,3}{3,4}{4}{5}{6,7}{8}>|<{3} {6}>|<font color='green'> **Yes**  </font>
<{1,3}{3,4}{4}{5}{6,7}{8}>|<{1,3} {6}>|<font color='red'> **No**  </font>


### Minimum & Maximum Gap

The gap is the time between two consecutive
elements.
> mingap of zero means elements must occur immediately after each other.

> Example: If mingap=1 and maxgap=3.

Sequence A |Sequence B |min gap | max gap
-----|-----|-----|---
<{1,3}{3,4}{4}{5}{6,7}{8}>|<{3} {6}>|<font color='green'> **PASS** </font>|<font color='green'> **PASS** </font>
<{1,3}{3,4}{4}{5}{6,7}{8}>|<{6} {8}>|<font color='red'> **Fail**  </font>|<font color='green'> **PASS** </font>
<{1,3}{3,4}{4}{5}{6,7}{8}>|<{1,3} {6}>|<font color='green'> **PASS**  </font>|<font color='red'> **Fail** </font>
<{1,3}{3,4}{4}{5}{6,7}{8}>|<{1} {3} {8}>|<font color='red'> **Fail**  </font>|<font color='red'> **Fail** </font>

In [9]:
def supports(sequence, cand_seq, max_span=np.inf, min_gap=0, max_gap=np.inf):
    for idx, event in enumerate(sequence):
        i = 0
        if set(event[1] if isinstance(event, tuple) else event).issuperset(cand_seq[i]):
            min_t = event[0] if isinstance(event, tuple) else idx
            i += 1

            # special case if cand_seq is a sequence of one element
            if i == len(cand_seq):
                return True

            prev_t = event[0] if isinstance(event, tuple) else idx

            for t, itemset in (sequence[idx + 1:] if isinstance(sequence[idx], tuple) 
                               else enumerate(sequence[idx + 1:], start=idx + 1)):

                # the min_gap constraint is violated
                if not t - prev_t > min_gap:
                    continue

                # the max_gap constraint is violated
                if not t - prev_t <= max_gap:
                    break

                # the max_span constraint is violated
                if t - min_t > max_span:
                    break

                if set(itemset).issuperset(cand_seq[i]):
                    i += 1

                # the sequence satisfies all the time constraints
                if i == len(cand_seq):
                    return True
    return False

In [10]:
sequence = [[1, 3], [3, 4], [4], [5], [6, 7], [8]]

In [11]:
print(supports(sequence, [[3], [4]], max_span=3))
print(supports(sequence, [[3], [6]], max_span=3))
print(supports(sequence, [[1, 3], [6]], max_span=3))

True
True
False


In [12]:
print(supports(sequence, [[3], [6]], min_gap=1))
print(supports(sequence, [[3], [6]], max_gap=3))

True
True


In [13]:
print(supports(sequence, [[6], [8]], min_gap=1))
print(supports(sequence, [[6], [8]], max_gap=3))

False
True


In [14]:
print(supports(sequence, [[1, 3], [6]], min_gap=1))
print(supports(sequence, [[1, 3], [6]], max_gap=3))

True
False


In [15]:
print(supports(sequence, [[1], [3], [8]], min_gap=1))
print(supports(sequence, [[1], [3], [8]], max_gap=3))

False
False


In [16]:
supports([[2, 4], [3, 5, 6], [4, 7], [4, 5], [8]], [[6], [5]], max_span=4, min_gap=0, max_gap=2)

True

In [17]:
supports([[1], [2], [3], [4], [5]], [[1], [4]], max_span=4, min_gap=0, max_gap=2)

False

In [18]:
supports([[1], [2, 3], [3, 4], [4, 5]], [[2], [3], [5]], max_span=4, min_gap=0, max_gap=2)

True

In [19]:
supports([[1, 2], [3], [2, 3], [3, 4], [2, 4], (6, [4, 5])], [[1, 2], [5]], max_span=4, min_gap=0, max_gap=2)

False

# Support Count

In [20]:
def count_support(dataset, cand_seq, max_span=np.inf, min_gap=0, max_gap=np.inf):
  
    if max_span == np.inf and min_gap == 0 and max_gap == np.inf: # no time constraints

        return sum(1 for seq in dataset if is_subsequence([event[1] for event in seq] if isinstance(seq[0], tuple) else seq, cand_seq))
    else:
        return sum(1 for seq in dataset if supports(seq, cand_seq, max_span, min_gap, max_gap))

In [21]:
count_support(dataset, [['b']])

5

In [22]:
count_support(dataset, [['a'], ['b', 'c']])

0

 ### Generate candidate sequences.
 

 
    Generates one candidate of length k from two candidates of length (k-1)



In [23]:
def gen_cands_for_pair(cand1, cand2):
    
    cand1_clone = copy.deepcopy(cand1)
    cand2_clone = copy.deepcopy(cand2)
    # drop the leftmost item from cand1:
    if len(cand1[0]) == 1:
        cand1_clone.pop(0)
    else:
        cand1_clone[0] = cand1_clone[0][1:]
    # drop the rightmost item from cand2:
    if len(cand2[-1]) == 1:
        cand2_clone.pop(-1)
    else:
        cand2_clone[-1] = cand2_clone[-1][:-1]
    
    # if the result is not the same, then we dont need to join
    if not cand1_clone == cand2_clone:
        return []
    else:
        new_cand = copy.deepcopy(cand1)
        if len(cand2[-1]) == 1:
            new_cand.append(cand2[-1])
        else:
            new_cand[-1].extend([cand2[-1][-1]])
        return new_cand

In [24]:
candA = [['a'], ['b', 'c'], ['d']]
candB = [['b', 'c'], ['d', 'e']]
gen_cands_for_pair(candA, candB)

[['a'], ['b', 'c'], ['d', 'e']]

In [25]:
candA = [['a'], ['b', 'c'], ['d']]
candC = [['b', 'c'], ['d'], ['e']]
gen_cands_for_pair(candA, candC)

[['a'], ['b', 'c'], ['d'], ['e']]

In [26]:
candA = [['a'], ['b', 'c'], ['d']]
candD = [['a'], ['b', 'c'], ['e']]
gen_cands_for_pair(candA, candD)

[]

 ### Generate candidate sequences.
 

 
    Generates the set of candidates of length k from the set of frequent sequences with length (k-1)

In [27]:
def gen_cands(last_lvl_cands):
    
    k = sequence_length(last_lvl_cands[0]) + 1
    if k == 2:
        flat_short_cands = [item for sublist2 in last_lvl_cands for sublist1 in sublist2 for item in sublist1]
        result = [[[a, b]] for a in flat_short_cands for b in flat_short_cands if b > a]
        result.extend([[[a], [b]] for a in flat_short_cands for b in flat_short_cands])
        return result
    else:
        cands = []
        for i in range(0, len(last_lvl_cands)):
            for j in range(0, len(last_lvl_cands)):
                new_cand = gen_cands_for_pair(last_lvl_cands[i], last_lvl_cands[j])
                if not new_cand == []:
                    cands.append(new_cand)
        cands.sort()
        return cands

Lets assume we know the frequent sequences of level 2:

In [28]:
last_lvl_freq_patterns = [
    
    [['a', 'b']], 
    [['b', 'c']], 
    [['a'], ['b']], 
    [['a'], ['c']], 
    [['b'], ['c']], 
    [['c'], ['b']], 
    [['c'], ['c']]
]

Then we can compute the generate candidates for level 3:

In [29]:
new_cands = gen_cands(last_lvl_freq_patterns)
new_cands

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['a', 'b', 'c']],
 [['b'], ['c'], ['b']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['b']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [30]:
def gen_direct_subsequences(sequence):
    
    result = []
    for i, itemset in enumerate(sequence):
        if len(itemset) == 1:
            seq_clone = copy.deepcopy(sequence)
            seq_clone.pop(i)
            result.append(seq_clone)
        else:
            for j in range(len(itemset)):
                seq_clone = copy.deepcopy(sequence)
                seq_clone[i].pop(j)
                result.append(seq_clone)
    return result

In [31]:
def gen_contiguous_direct_subsequences(sequence):
    
    result = []
    for i, itemset in enumerate(sequence):
        # first or last element
        if i == 0 or i == len(sequence) - 1:
            if len(itemset) == 1:
                seq_clone = copy.deepcopy(sequence)
                seq_clone.pop(i)
                result.append(seq_clone)
            else:
                for j in range(len(itemset)):
                    seq_clone = copy.deepcopy(sequence)
                    seq_clone[i].pop(j)
                    result.append(seq_clone)
        else:  # middle element
            if len(itemset) > 1:
                for j in range(len(itemset)):
                    seq_clone = copy.deepcopy(sequence)
                    seq_clone[i].pop(j)
                    result.append(seq_clone)
    return result

# Candidate Pruning.

In [32]:
def prune_cands(last_lvl_cands, cands_gen, max_gap=np.inf):
       
    return [cand for cand in cands_gen if all(x in last_lvl_cands for x in (gen_contiguous_direct_subsequences(cand) if max_gap != np.inf 
                                                                            else gen_direct_subsequences(cand)))]

In [33]:
cands_pruned = prune_cands(last_lvl_freq_patterns, new_cands)
cands_pruned

[[['a'], ['b'], ['c']],
 [['a'], ['b', 'c']],
 [['a'], ['c'], ['b']],
 [['a'], ['c'], ['c']],
 [['a', 'b'], ['c']],
 [['b'], ['c'], ['c']],
 [['b', 'c'], ['c']],
 [['c'], ['b'], ['c']],
 [['c'], ['b', 'c']],
 [['c'], ['c'], ['b']],
 [['c'], ['c'], ['c']]]

In [34]:
min_sup = 0.4
cands_counts = [(s, count_support(dataset, s)) for s in cands_pruned]
result_lvl = [(i, count) for i, count in cands_counts if count >= min_sup * len(dataset)]
result_lvl

[]

In [35]:
def gsp(dataset, min_sup, max_span=np.inf, min_gap=0, max_gap=np.inf, verbose=False):
    
    overall = []
    min_sup *= len(dataset)
    # make the first pass over the sequence database to yield all the 1-element frequent subsequences
    items = sorted(set([item for sequence in dataset
                        for event in sequence
                        for item in (event[1] if isinstance(event, tuple) else event)]))
    single_item_sequences = [[[item]] for item in items]
    single_item_counts = [(s, count_support(dataset, s)) for s in single_item_sequences]
    single_item_counts = [(i, count) for i, count in single_item_counts if count >= min_sup]
    overall.append(single_item_counts)
    if verbose > 0:
        print('Result, lvl 1: ' + str(overall[0]))
        print('--------------------------------------------------------------------------------------------------')
    k = 1
    while overall[k - 1]:
        # 1. candidate generation: merge pairs of frequent subsequences found in the 
        # (k-1)th pass to generate candidate sequences that contain k items 
        last_lvl_cands = [x[0] for x in overall[k - 1]]
        cands_gen = gen_cands(last_lvl_cands)
        # 2. candidate pruning: prune candidate k-sequences that contain infrequent 
        # (contiguous) (k-1)-subsequences (Apriori principle)
        cands_pruned = prune_cands(last_lvl_cands, cands_gen, max_gap)
        # 3. support counting: make a new pass over the sequence database to find 
        # the support for these candidate sequences
        cands_counts = [(s, count_support(dataset, s, max_span, min_gap, max_gap)) for s in cands_pruned]
        # 4. candidate elimination: eliminate candidate k-sequences whose actual 
        # support is less than `minsup`
        result_lvl = [(i, count) for i, count in cands_counts if count >= min_sup]
        if verbose > 1:
            
            print('Result, LeveL ' + str(k + 1) + ': ' + str(result_lvl))
            print('--------------------------------------------------------------------------------------------------')   
            if verbose > 1:
                print('Candidates generated, lvl ' + str(k + 1) + ': ' + str(cands_gen))
                print('\n') 
                print('Candidates pruned, lvl ' + str(k + 1) + ': ' + str(cands_pruned))
                
        overall.append(result_lvl)
        k += 1
    # "flatten" overall
    overall = overall[:-1]
    overall = [item for sublist in overall for item in sublist]
    overall.sort(key=lambda tup: (tup[1], neg(sequence_length(tup[0]))), reverse=True)
    return overall

### LeveL 1 (Frequent 1-sequences)

| Event | count |
|:------------:|-----------:|
| a | 2 |
| b | 5|
| c | 4 |
| d | 2|
| e | 2|
| f | 2|

### LeveL 2 (Frequent 2-sequences)

| Seq | count |
|:------------:|-----------:|
| {b}{b} | 3 |
| {a}{b} | 2|
| {b}{c}| 4 |
| {c}{b} | 3|
| {bd} | 2|
| {d}{b} | 2|
| {a}{b} | 2|
| {b}{e} | 2|
| {d}{c} | 2|
| {ce} | 2|

### LeveL 3 (Frequent 3-sequences)

| Event | count |
|:------------:|-----------:|
| {b}{c}{b} | 3 |
| {b}{ce} | 2|
| {bd}{b} | 2 |
| {bd}{c} | 2|
| {d}{c}{b} | 2|


### LeveL 4 (Frequent 4-sequences)

| Event | count |
|:------------:|-----------:|
| {bd}{c}{b} | 2 |



In [36]:
gsp(dataset, min_sup=0.4, verbose=2)

Result, lvl 1: [([['a']], 2), ([['b']], 5), ([['c']], 4), ([['d']], 2), ([['e']], 2), ([['f']], 2)]
--------------------------------------------------------------------------------------------------
Result, LeveL 2: [([['b', 'd']], 2), ([['b', 'f']], 2), ([['c', 'e']], 2), ([['a'], ['b']], 2), ([['b'], ['b']], 3), ([['b'], ['c']], 4), ([['b'], ['e']], 2), ([['c'], ['b']], 3), ([['d'], ['b']], 2), ([['d'], ['c']], 2)]
--------------------------------------------------------------------------------------------------
Candidates generated, lvl 2: [[['a', 'b']], [['a', 'c']], [['a', 'd']], [['a', 'e']], [['a', 'f']], [['b', 'c']], [['b', 'd']], [['b', 'e']], [['b', 'f']], [['c', 'd']], [['c', 'e']], [['c', 'f']], [['d', 'e']], [['d', 'f']], [['e', 'f']], [['a'], ['a']], [['a'], ['b']], [['a'], ['c']], [['a'], ['d']], [['a'], ['e']], [['a'], ['f']], [['b'], ['a']], [['b'], ['b']], [['b'], ['c']], [['b'], ['d']], [['b'], ['e']], [['b'], ['f']], [['c'], ['a']], [['c'], ['b']], [['c'], ['c']], 

[([['b']], 5),
 ([['c']], 4),
 ([['b'], ['c']], 4),
 ([['b'], ['b']], 3),
 ([['c'], ['b']], 3),
 ([['b'], ['c'], ['b']], 3),
 ([['a']], 2),
 ([['d']], 2),
 ([['e']], 2),
 ([['f']], 2),
 ([['b', 'd']], 2),
 ([['b', 'f']], 2),
 ([['c', 'e']], 2),
 ([['a'], ['b']], 2),
 ([['b'], ['e']], 2),
 ([['d'], ['b']], 2),
 ([['d'], ['c']], 2),
 ([['b'], ['c', 'e']], 2),
 ([['b', 'd'], ['b']], 2),
 ([['b', 'd'], ['c']], 2),
 ([['d'], ['c'], ['b']], 2),
 ([['b', 'd'], ['c'], ['b']], 2)]